In [5]:
import itertools
import numpy as np
import math
import torch
from torch.autograd import Variable

# (a), (b)

In [38]:
ppl = ['SR','YD','MG','ZH','HS','RS','NZ','YK']

In [21]:
unarypotentials = [[0,0,0,0,0,0,0,0],[2,-2,-2,-8,-2,3,-2,1]]

In [40]:
def unary_potential(val,ind):
    return unarypotentials[val][ind]

In [41]:
def binary_potential(val1,val2):
    if val1 == val2 == 1:
        return 2
    else:
        return 0

In [42]:
def global_score(SGvec):
    UPs = 0
    for i in range(0,8):
        UPs += unary_potential(SGvec[i],i)
        
    BPs = binary_potential(SGvec[1],SGvec[5])+binary_potential(SGvec[2],SGvec[5])+binary_potential(SGvec[5],SGvec[3])+binary_potential(SGvec[3],SGvec[7])+binary_potential(SGvec[3],SGvec[4])+binary_potential(SGvec[3],SGvec[6])
    return math.exp(UPs + BPs)

In [43]:
def enum_log_partition(unarypotentials):
    normterm = 0
    for j in itertools.product(range(0,2),repeat=8):
        UPs = 0
        for i in range(0,8):
            seen = j[i]
            UPs += unarypotentials[seen][i]

        BPs = binary_potential(j[1],j[5])+binary_potential(j[2],j[5])+binary_potential(j[5],j[3])+binary_potential(j[3],j[7])+binary_potential(j[3],j[4])+binary_potential(j[3],j[6])
        normterm += math.exp(UPs + BPs)
        
    return normterm

In [44]:
global_score([1,0,0,0,0,1,0,0])

148.4131591025766

In [45]:
enum_log_partition(unarypotentials)

3423.3716456041548

In [46]:
math.log(enum_log_partition(unarypotentials))

8.138381205702334

In [47]:
global_score([1,0,0,0,0,1,0,0])/enum_log_partition(unarypotentials)

0.04335292059018755

# (c)

In [48]:
def pRS_seen(unarypotentials):
    normterm = enum_log_partition(unarypotentials)
    
    probRS_seen = 0
    
    for j in itertools.product(range(0,2),repeat=7):
        j = [1,j[0],j[1],j[2],j[3],j[4],j[5],j[6]]
        UPs = 0
        for i in range(0,8):
            seen = j[i]
            UPs += unarypotentials[seen][i]

        BPs = binary_potential(j[1],j[5])+binary_potential(j[2],j[5])+binary_potential(j[5],j[3])+binary_potential(j[3],j[7])+binary_potential(j[3],j[4])+binary_potential(j[3],j[6])
        probRS_seen += math.exp(UPs + BPs)
        
    return probRS_seen/normterm

In [49]:
pRS_seen(unarypotentials)

0.8807970779778823

# (d)

# (e)

In [145]:
bigtheta = Variable(torch.Tensor([0,0,0,2,0,0,0,2,0,0,0,2,0,0,0,2,0,0,0,2,0,0,0,2,unarypotentials[0][0],unarypotentials[1][0],unarypotentials[0][1],unarypotentials[1][1],unarypotentials[0][2],unarypotentials[1][2],unarypotentials[0][3],unarypotentials[1][3],unarypotentials[0][4],unarypotentials[1][4],unarypotentials[0][5],unarypotentials[1][5],unarypotentials[0][6],unarypotentials[1][6],unarypotentials[0][7],unarypotentials[1][7]]),requires_grad = True)

In [146]:
def score_assignment(j,phi):
    if j[2] == j[5] == 0:
        phi[0] = 1
    elif j[2] == 0 and j[5] == 1:
        phi[1] = 1
    elif j[2] == 1 and j[5] == 0:
        phi[2] = 1
    else:
        phi[3] = 1
            
    if j[1] == j[5] == 0:
        phi[4] = 1
    elif j[1] == 0 and j[5] == 1:
        phi[5] = 1
    elif j[1] == 1 and j[5] == 0:
        phi[6] = 1
    else:
        phi[7] = 1
            
    if j[5] == j[3] == 0:
        phi[8] = 1
    elif j[5] == 0 and j[3] == 1:
        phi[9] = 1
    elif j[5] == 1 and j[3] == 0:
        phi[10] = 1
    else:
        phi[11] = 1
        
    if j[3] == j[7] == 0:
        phi[12] = 1
    elif j[3] == 0 and j[7] == 1:
        phi[13] = 1
    elif j[3] == 1 and j[7] == 0:
        phi[14] = 1
    else:
        phi[15] = 1
        
    if j[3] == j[6] == 0:
        phi[16] = 1
    elif j[3] == 0 and j[6] == 1:
        phi[17] = 1
    elif j[3] == 1 and j[6] == 0:
        phi[18] = 1
    else:
        phi[19] = 1
            
    if j[3] == j[4] == 0:
        phi[20] = 1
    elif j[3] == 0 and j[4] == 1:
        phi[21] = 1
    elif j[3] == 1 and j[4] == 0:
        phi[22] = 1
    else:
        phi[23] = 1
            
    for k in range(0,8):
        if j[k] == 0:
            phi[24+2*k] = 1
        else:
            phi[24+2*k+1] = 1
    
    return phi

def e_enum_log_partition(ranges):
    sumstuff = []
#     normterm_e = Variable(torch.Tensor([0]),requires_grad = True)
    for j in ranges:
        phi = np.zeros(len(bigtheta.data.numpy()))
        
        phi = score_assignment(j,phi)
        
        phi = Variable(torch.Tensor(phi))
            
        sumstuff.append(torch.exp(torch.dot(phi,bigtheta)))
        
    return torch.log(torch.cat(sumstuff,0).sum(0))

In [147]:
aofbigtheta = e_enum_log_partition(itertools.product(range(0,2),repeat=8))
aofbigtheta

Variable containing:
 8.1384
[torch.FloatTensor of size 1]

In [148]:
e_enum_log_partition([[1,0,0,0,0,1,0,0]])

Variable containing:
 5
[torch.FloatTensor of size 1]

In [149]:
aofbigtheta.backward()

In [150]:
bigtheta.grad

Variable containing:
 0.0134
 0.4924
 0.0018
 0.4924
 0.0134
 0.4924
 0.0018
 0.4924
 0.0151
 0.0001
 0.9436
 0.0412
 0.2578
 0.7009
 0.0020
 0.0393
 0.8445
 0.1143
 0.0206
 0.0206
 0.8445
 0.1143
 0.0206
 0.0206
 0.1192
 0.8808
 0.5058
 0.4942
 0.5058
 0.4942
 0.9588
 0.0412
 0.8651
 0.1349
 0.0152
 0.9848
 0.8651
 0.1349
 0.2598
 0.7402
[torch.FloatTensor of size 40]

# (f)

In [87]:
order = [2,5,1,3,6,7,4,0] # MG, RS, YD, ZH, NZ, YK, HS, SR (unconnected)
backwards = [0,4,7,6,4,1,5,2]

In [88]:
arr_child = [[], [], [5], [6, 4, 7], [], [3, 1], [], []]
arr_par = [None, 5, None, 5, 3, 2, 3, 3]

In [98]:
beliefs = np.array([[0.0]*8,[0.0]*8])
mges = np.array([[[0.0]*8]*8,[[0.0]*8]*8])

In [109]:
for node in backwards:
    for seen in range(0,2):
        beliefs[seen][node] = np.exp(unarypotentials[seen][node])*np.prod([mges[seen][child][node] for child in arr_child[node]])
    
    normterm = beliefs[0][node]+beliefs[1][node]
    print beliefs[0][node],beliefs[1][node]
    print beliefs[0][node]/(normterm+0.0),beliefs[1][node]/(normterm+0.0)
    beliefs[0][node] = beliefs[0][node]/(normterm)
    beliefs[1][node] = beliefs[1][node]/(normterm)
    
    if arr_par[node]:
        mges[0][node][arr_par[node]] = 1
        mges[1][node][arr_par[node]] = math.exp(2)*beliefs[1][node] + beliefs[0][node]

In [102]:
rootbelief = np.array([[0.0]*8,[0.0]*8])
node0 = backwards[-1]
for seen in range(0,2):
    rootbelief[seen][node0] = np.exp(unarypotentials[seen][node0])*np.prod([mges[seen][child][node0] for child in arr_child[node0]])

normterm0 = rootbelief[0][node0]+rootbelief[1][node]
rootbelief[0][node0] = rootbelief[0][node0]/(normterm0+0.0)
rootbelief[1][node0] = rootbelief[1][node0]/(normterm0+0.0)

In [103]:
mges_down = np.array([[[0.0]*8]*8,[[0.0]*8]*8])

In [104]:
for node in order:
    if arr_par[node]:
        for seen in range(0,2):
            rootbelief[seen][node] = beliefs[seen][node]*mges_down[seen][arr_par[node]][node]
    else:
        for seen in range(0,2):
            fracterm = np.exp(unarypotentials[seen][node])
            rootbelief[seen][node] = math.exp(fracterm)/(math.exp(fracterm+1)+0.0)
            
    normtermdown = rootbelief[0][node]+rootbelief[1][node]
    rootbelief[0][node] = rootbelief[0][node]/(normtermdown+0.0)
    rootbelief[1][node] = rootbelief[1][node]/(normtermdown+0.0)
    
    for child in arr_child[node]:
        mges_down[0][node][child] = (rootbelief[0][node]/(mges[0][child][node]+0.0)) + (rootbelief[1][node]/(mges[1][child][node]+0.0))
        mges_down[1][node][child] = (rootbelief[0][node]/(mges[0][child][node]+0.0)) + math.exp(2)*(rootbelief[1][node]/(mges[1][child][node]+0.0))